In [0]:
%run ./_common_helpers

In [0]:
# Widgets
dbutils.widgets.text("include_existing", "false") # set to true for the first time, false for subsequent runs
dbutils.widgets.text("trigger", "once") # set to a time value so processingTime picks it up and runs continuously

# Parameters
include_existing = dbutils.widgets.get("include_existing").lower()
trigger_str = dbutils.widgets.get("trigger")

In [0]:
# Paths
base_path = "/Volumes/claims360_dev/bronze"
read_path = f"{base_path}/raw/remit_835"

checkpoint_location = f"{base_path}/ingestion/_checkpoints/remit_835"
auto_loader_schema_location = f"{base_path}/ingestion/_schemas/remit_835"

bronze_table = "claims360_dev.bronze.payments_835_raw"

In [0]:
# Reader
src_df = (spark.readStream
          .format("cloudFiles")
          .option("cloudFiles.format", "json")
          .option("multiLine", "true")
          .option("cloudFiles.schemaLocation", auto_loader_schema_location)
          .option("cloudFiles.schemaEvolutionMode", "rescue")
          .option("cloudFiles.includeExistingFiles", include_existing)
          .option("cloudFiles.rescuedDataColumn", "_rescued")
          .load(read_path))

In [0]:
# Add metadata
bronze_df = add_ingest_metadata(src_df, "835")

In [0]:
# Writer
(bronze_df.writeStream
    .format("delta")
    .outputMode("append")
    .trigger(availableNow=True)
    .option("checkpointLocation", checkpoint_location)
    .option("mergeSchema", "true")
    .toTable(bronze_table))

In [0]:
%sql
SELECT * FROM claims360_dev.bronze.ehr_claims_raw;